In [6]:
import pandas as pd
import logging

In [2]:
from dotenv import load_dotenv
load_dotenv(".flaskenv")

import registration
from registration import db
from registration.models import *

app = registration.create_app()
app.app_context().push()

In [32]:
df = pd.read_excel("../202402_StämmeBdP-PfadiTag.XLS", converters={'Nummer': str, "Ort": str})
df["Ort"] = df["Ort"].fillna("")
df[df["Ort"].isna()]

,Nummer,Landesverband,Stamm,Ort


In [4]:
bdp = db.session.get(Group, "bdp")
if not bdp:
    bdp = Group()
    bdp.id = "bdp"
    bdp.name = "Bund der Pfadfinderinnen und Pfadfinder"
    db.session.add(bdp)
    db.session.commit()
bdp

<Group bdp (Bund der Pfadfinderinnen und Pfadfinder)>

In [34]:
for name, row in df.iterrows():
    land_id = "bdp_" + row["Nummer"][0:4] + "000000"
    group_id = "bdp_" + row["Nummer"]
    
    land = db.session.get(Group, land_id)
    if not land:
        land = Group()
        land.id = land_id
        land.name = row["Landesverband"]
        land.name = land.name[14:] if land.name.startswith("Landesverband ") else land.name
        land.group_type = "Landesverband"
        land.parent_id = bdp.id
        db.session.add(land)
        db.session.commit()
        logging.warning("Created land %s", land)
    
    group = db.session.get(Group, group_id)
    if not group:
        group = Group()
        group.id = group_id
        group.parent_id = land.id
        group.group_type, group.name = row["Stamm"].split(" ", 1)
        group.city = row["Ort"]
        db.session.add(group)
        db.session.commit()
        logging.warning("Created group %s", group)
